# Test Time Augmentation

In [46]:
import pandas as pd
import torch
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')
from dataset import AudioDataset
from model import AlexNet

In [3]:
test_df = pd.read_csv('../data/sample_submission.csv')

In [4]:
test_df.head()

,fname,label
0,00063640.wav,Laughter Hi-Hat Flute
1,0013a1db.wav,Laughter Hi-Hat Flute
2,002bb878.wav,Laughter Hi-Hat Flute
3,002d392d.wav,Laughter Hi-Hat Flute
4,00326aa9.wav,Laughter Hi-Hat Flute


In [35]:
test_dataset = AudioDataset(test_df[:10], '../data/audio_test', test=True)

In [36]:
print(len(test_dataset))

10


In [37]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [38]:
print(len(test_loader))

1


In [39]:
num_classes = 41
model = AlexNet(num_classes)

In [40]:
model

AlexNet(
  (block1): Sequential(
    (0): Conv2d(1, 48, kernel_size=(11, 11), stride=(2, 3), padding=(5, 5))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): Sequential(
    (0): Conv2d(48, 128, kernel_size=(5, 5), stride=(2, 3), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block3): Sequential(
    (0): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(192, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (7)

In [41]:
model.load_state_dict(torch.load('../logs/log.020/epoch102-1.703-0.765.pth', map_location=lambda storage, loc: storage))

In [42]:
model.eval()

AlexNet(
  (block1): Sequential(
    (0): Conv2d(1, 48, kernel_size=(11, 11), stride=(2, 3), padding=(5, 5))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): Sequential(
    (0): Conv2d(48, 128, kernel_size=(5, 5), stride=(2, 3), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block3): Sequential(
    (0): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(192, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (7)

In [54]:
num_aug = 5

with torch.no_grad():
    tta_predictions = None
    for i in range(num_aug):
        predictions = []
        for batch_idx, data in enumerate(test_loader):
            output = model(data)
            predictions.append(output)
        predictions = torch.cat(predictions, dim=0)
        if tta_predictions is None:
            tta_predictions = np.zeros_like(predictions)
        tta_predictions += predictions
        print(i, predictions.size(), predictions[0][:5])

    tta_predictions /= num_aug
    print(tta_predictions[0][:5])

0 torch.Size([10, 41]) tensor([ -7.5326, -19.0391, -12.1112,   1.0037, -11.0376])
1 torch.Size([10, 41]) tensor([ -8.3328, -20.4006, -14.3846,   3.5634, -12.1790])
2 torch.Size([10, 41]) tensor([ -9.0892, -21.8444, -16.0997,   1.7139, -11.4592])
3 torch.Size([10, 41]) tensor([ -6.1393, -20.3961, -11.7629,  -0.2023,  -9.4429])
4 torch.Size([10, 41]) tensor([ -4.2086, -19.9219, -11.1338,   2.8850,  -6.9384])
[ -7.0605025 -20.320438  -13.098444    1.7927482 -10.211402 ]
